# RNN 중간 출력 구조
- RNN cell이 2개 있는 구조에서 중간출력 y값 도출하는 코드

## Wy = Wh 같다고 가정한 구조

In [27]:
import numpy as np
import tensorflow as tf
# tensorflow가 만든 graph 초기화
tf.reset_default_graph()

# X입력 데이터 요소 개수
n_inputs = 3
# 하나의 RNN cell안에 들어있는 neuron 개수
n_neurons = 5

X0 = tf.placeholder(tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(tf.float32, shape=[None, n_inputs])

# X입력 데이터가 RNN cell로 들어가서, 입력 데이터와 뉴런 사이의 Weight
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
# RNN Cell안에 있는 뉴런과 해당 뉴런개수만큼 출력값 사이의 Weight
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
# 중간 출력값 y에다가 더해줄 bias(어차피 더하는 거라서 1차원, 2차원이여도 노상관!? 시도해보기)
b = tf.Variable(tf.zeros(shape=[1, n_neurons]))

# X0 size: 데이터개수 by *n_inputs* / Wx size: *n_inputs* by 뉴런개수
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
# tf.matmul(Y0, Wy)-> 원래는 Wh에 해당함!
# tf.matmul(X1, Wx) + b : 새로운 인풋에 대해 RNN Cell안에서 출력될 Output 
# 위 2개를 더해서 state를 참조한 최종 output 출력
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

X0_batch = np.array([[0,1,2],
                    [3,4,5],
                    [6,7,8],
                    [9,0,1]])
X1_batch = np.array([[9,8,7],
                    [3,4,5],
                    [6,5,4],
                    [3,2,1]])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})

print(f"Y0_val:{Y0_val.shape}\n{Y0_val}")
print()
print(f"Y1_val:{Y1_val.shape}\n{Y1_val}")

Y0_val:(4, 5)
[[ 0.9561854  -0.39263886  0.9843402  -0.99443775  0.9188682 ]
 [ 0.9992524   0.3020234   0.9999655  -0.9999992   0.4067599 ]
 [ 0.9999878   0.7772576   1.         -1.         -0.61583143]
 [-0.9999978   0.99996006  0.999072   -0.99346554 -0.7945948 ]]

Y1_val:(4, 5)
[[ 0.9997118   0.99866587  0.99978536 -1.         -0.9979562 ]
 [ 0.9966941   0.9422897   0.9960871  -0.999998    0.7939893 ]
 [-0.32207626  0.9972328   0.30107394 -0.9999929  -0.999524  ]
 [-0.9992126  -0.28566796 -0.05459739 -0.9998848  -0.99297464]]


In [3]:
# 위 구조를 RNN 함수를 사용해 구현
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings(action='ignore')

tf.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(tf.float32, shape=[None, n_inputs])

# num_units : RNN Cell안에 들어있는 노드 개수
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)

# static_rnn: 여러 RNN cell을 이어주는 역할
# states : 마지막 states만 출력 = 최종 Output과 동일       #입력데이터
# output_seqs : 각 RNN cell에서 출력되는 y값들
output_seqs, states = tf.nn.static_rnn(cell=basic_cell, inputs=[X0, X1], dtype=tf.float32)

Y0, Y1 = output_seqs

X0_batch = np.array([[0,1,2],
                    [3,4,5],
                    [6,7,8],
                    [9,0,1]])
X1_batch = np.array([[9,8,7],
                    [3,4,5],
                    [6,5,4],
                    [3,2,1]])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

print(f"Y0_val:{Y0_val.shape}\n{Y0_val}")
print()
print(f"Y1_val:{Y1_val.shape}\n{Y1_val}")

Y0_val:(4, 5)
[[-0.7765463  -0.17783469  0.94708014 -0.24107334  0.65518   ]
 [-0.94404536 -0.9784627   0.999941   -0.5216255   0.9448531 ]
 [-0.98691344 -0.99966055  1.         -0.7217208   0.9923108 ]
 [ 0.98980343 -0.9998622  -0.06139848  0.9953281   0.9693675 ]]

Y1_val:(4, 5)
[[-0.61016476 -0.9999443   0.9999999  -0.51656985  0.96545655]
 [-0.49017042 -0.8415049   0.99997324 -0.6579212   0.9188109 ]
 [ 0.7475013  -0.9937336   0.9999706  -0.60205674  0.81786746]
 [-0.164256   -0.99079937  0.74256796  0.02382644  0.06473937]]


## 시간축, 샘플축이 한 array에 있을 경우

In [4]:
tf.reset_default_graph()

# 시간축 즉, RNN cell을 몇 개할 건지!
n_steps = 2
# 입력데이터 안의 요소 개수
n_inputs = 3
# RNN cell안의 노드 개수
n_neurons = 5

# shape=[데이터개수, RNN개수, 입력데이터사이즈]
X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])
'''
perm=[0,1,2] -> perm=[1,0,2] 
- 0: 샘플개수 축
- 1: 시간축(RNN cell)
- 2: 요소개수 축
=> 시간축을 가장 바깥, 그 다음엔 샘플개수 축, 마지막에 요소개수 축으로 변환(why?시간축(RNN cell) 단위로 샘플을 나누도록 하기 위해)(transpose)
=> 그러고 시간축으로 나눈 샘플을 unstack으로 시간축 단위 덩어리로 쪼개기!
'''
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
# RNN Cell안에 노드 개수 설정해 만들기
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# static_rnn으로 RNN cell이어 붙여서 입력 데이터(샘플) 넣어서 중간 y값들과 최종 states(=최종output) 출력
outputs_seqs, states = tf.nn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
## unstack된 출력값들 stack시키고 transpose시키기 for 원본 X_batch와 동일한 형태로 만들기 위해!
outputs = tf.transpose(tf.stack(outputs_seqs), perm=[1,0,2])

# Sample이 t축보다 더 바깥에 있음-> t축을 Sample축보다 바깥으로 빼내야함!
X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [3,4,5]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    # 출력값 도출하기 위해 쌓은 RNN 텐서들 실행
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    
print(outputs_val.shape,'\n', outputs_val)

(4, 2, 5) 
 [[[ 0.78218913  0.6348658   0.7371695   0.11419411  0.8202143 ]
  [ 0.9999997   0.9980055   0.9999993   0.9999995  -0.9108817 ]]

 [[ 0.9983674   0.9527139   0.99826217  0.990955    0.7456397 ]
  [ 0.99995637  0.997671    0.9997526   0.9842986   0.8610077 ]]

 [[ 0.99998915  0.99476254  0.99999017  0.99994814  0.6461452 ]
  [ 0.9999807   0.99467796  0.9999352   0.99983656 -0.8338219 ]]

 [[ 0.9370286   0.9861273   0.9999709   0.99958795 -0.99973536]
  [ 0.99185     0.8628038   0.95696604  0.7895571  -0.8245704 ]]]


## dynamic_rnn = 입력데이터 시간축으로 unstack + static_rnn + 아웃풋 stack까지 한번에!

In [5]:
tf.reset_default_graph()

# 시간축 즉, RNN cell을 몇 개할 건지!
n_steps = 2
# 입력데이터 안의 요소 개수
n_inputs = 3
# RNN cell안의 노드 개수
n_neurons = 5

# [데이터개수, rnn개수(시간축), 입력데이터요소개수]
X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])
# RNN cell안의 노드개수 설정
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# 인자로 이어붙일 RNN cell, 입력 데이터샘플들 할당해 outputs와 최종 state출력
# dynamic_rnn = static_rnn + stack, Transpose 같이 수행해줌! 
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

# Sample이 t축보다 더 바깥에 있음-> t축을 Sample축보다 바깥으로 빼내야함!
X_batch = np.array([
      #t=0      t=1
    [[0,1,2], [9,8,7]],#sample1
    [[3,4,5], [3,4,5]],#sample2
    [[6,7,8], [6,5,4]],#sample3
    [[9,0,1], [3,2,1]] #sample4
])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})
print(outputs_val.shape, '\n', outputs_val)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
(4, 2, 5) 
 [[[-0.33220062 -0.07302813 -0.9540005   0.66247773 -0.6682915 ]
  [ 0.99993765 -0.9999713  -0.999999    0.9999978  -0.99999976]]

 [[ 0.8900091  -0.9708734  -0.999886    0.99688625 -0.99935335]
  [ 0.3653118  -0.99574953 -0.99989974  0.99930775 -0.99986786]]

 [[ 0.99661034 -0.9994944  -0.99999964  0.999976   -0.9999989 ]
  [ 0.992834   -0.9999174  -0.9995608   0.9998712  -0.99999523]]

 [[ 0.9995404  -0.999975    0.9358188   0.9993382  -0.9952824 ]
  [ 0.97126764 -0.9995928  -0.86882997  0.9931617  -0.99896574]]]


## 데이터 샘플의 길이를 가변적으로 처리하는 dynamic_rnn

In [15]:
tf.reset_default_graph()

# 시간축 즉, RNN cell을 몇 개할 건지!
n_steps = 2
# 입력데이터 안의 요소 개수
n_inputs = 3
# RNN cell안의 노드 개수
n_neurons = 5

X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])
# 1차원 형태로 데이터 샘플 중 어떤 것을 날릴지 담을 seq_length
seq_length = tf.placeholder(tf.int32, shape=[None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# 위의 RNN cell 연결하고 입력 데이터를 넣는데, 가변적으로 설정한 데이터 샘플 넣는 텐서
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32,
                                   sequence_length=seq_length)

X_batch = np.array([
    # t=0      t=1
    [[0,1,2], [9,8,7]],
    [[3,4,5], [0,0,0]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

'''
가변적인 데이터 샘플 정의
- array요소 개수 = 시간축 상관없이 각 들어가는 데이터 샘플 개수
- 해당 예시에서 2면 t=0, t=1모두 사용, 1이면 우선 0인 데이터샘플 사용안한다는 의미..둘 다 0이 아닌 값들로 채워져있을 때 1로설정시 시간기준으로 앞시간대의 샘플 날림!
'''
seq_length_batch = np.array([2, 1, 2, 2])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val, states_val = sess.run([outputs, states],
                                      feed_dict={X: X_batch, seq_length: seq_length_batch})

print(outputs_val.shape, '\n', outputs_val)
print('-'*70)
print(states_val.shape, '\n', states_val)
# states_val: 최종 state임. 즉 마지막 W_h값임!

(4, 2, 5) 
 [[[ 0.696327   -0.64561385  0.02407705 -0.7397376   0.76071125]
  [ 0.1570339  -1.          0.986322   -0.9996665   0.99983597]]

 [[ 0.7704643  -0.9996835   0.817209   -0.99367857  0.9953592 ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.8283338  -0.99999976  0.9789883  -0.99986565  0.9999204 ]
  [ 0.60398465 -0.9999926   0.7555552  -0.9352769   0.9661483 ]]

 [[-0.9985301  -0.9998438   0.83898634 -0.9609292   0.66543984]
  [-0.51347727 -0.99643934 -0.16703583 -0.49177277  0.45011327]]]
----------------------------------------------------------------------
(4, 5) 
 [[ 0.1570339  -1.          0.986322   -0.9996665   0.99983597]
 [ 0.7704643  -0.9996835   0.817209   -0.99367857  0.9953592 ]
 [ 0.60398465 -0.9999926   0.7555552  -0.9352769   0.9661483 ]
 [-0.51347727 -0.99643934 -0.16703583 -0.49177277  0.45011327]]


## Case1

In [3]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

hidden_size = 2
# RNN cell
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

# input : 4개요소, RNN cell 1개, Sample 1개
x_data = np.array([[[1,0,0,0]]], dtype=np.float32)
# output : 
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(outputs.eval())

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[[[ 0.33831435 -0.0132777 ]]]


## Case2

In [6]:
tf.reset_default_graph()

hidden_size = 2
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)

x_data = np.array([[[1,0,0,0],
                   [0,1,0,0],
                   [0,0,1,0],
                   [0,0,1,0],
                   [0,0,0,1]]],
                 dtype=np.float32)

outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(outputs.eval(),'\n')

[[[ 0.08892741  0.47140032]
  [-0.5325416   0.3200522 ]
  [-0.41576487  0.075031  ]
  [-0.38544068 -0.06497755]
  [ 0.4141925   0.36334524]]] 



## Case3

In [8]:
tf.reset_default_graph()

hidden_size = 4
n_inputs = 4
batch_size = 1
# n_steps = sequence_length: RNN cell의 개수
sequence_length = 4

idx2char = ['h','e','l','o']
# x_data의 요소숫자는 idx2char의 index에 해당!
x_data = [[0,1,2,2]]
x_one_hot = [[[1,0,0,0],
             [0,1,0,0],
             [0,0,1,0],
             [0,0,1,0]]]
# y_data는 원-핫 벡터를 인덱스로 표현한 것. ex) 1 => [0,1,0,0] 3 => [0,0,0,1] 
y_data = [[1,2,2,3]]

X = tf.placeholder(tf.float32, shape=[None, sequence_length, n_inputs])
# Y값 개수는 RNN cell 개수만큼 출력되니 sequence_length 만큼!
# 추후에 loss계산 시 정답과 비교해야 하므로 Y값 dtype을 int형으로 설정!(float형으로 하면 에러 발생)
Y = tf.placeholder(tf.int32, shape=[None, sequence_length])

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
# 초기 state 설정
initial_state = cell.zero_state(batch_size, tf.float32)
# 초기 state를 설정한 후 RNN cell들을 서로 연결해서 outputs 출력!
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

# 각 중간출력 y값들의 Loss를 종합할 때 곱할 Weight
weights = tf.ones([batch_size, sequence_length])
# targets에 들어가 있는 Y들과 RNN cell들이 만들어낸 y예측값들과 비교해 각 loss출력!
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
# 위에서 계산한 loss들 종합하기(평균내기)
loss = tf.reduce_mean(sequence_loss)
# SGD
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
# outputs 각 array에서 가장 큰 값을 갖는 index!!!를 반환.. 이 index값이 y_data(정답)과 일치하는지 보기!
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, 'loss:',l,'prediction: ', result, 'True Y: ', y_data)

0 loss: 1.71386 prediction:  [[2 3 2 1]] True Y:  [[1, 2, 2, 3]]
1 loss: 1.4416823 prediction:  [[1 1 2 3]] True Y:  [[1, 2, 2, 3]]
2 loss: 1.2793231 prediction:  [[1 1 2 3]] True Y:  [[1, 2, 2, 3]]
3 loss: 1.1567321 prediction:  [[1 1 2 3]] True Y:  [[1, 2, 2, 3]]
4 loss: 1.0519112 prediction:  [[1 1 2 3]] True Y:  [[1, 2, 2, 3]]
5 loss: 0.9852764 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
6 loss: 0.92299926 prediction:  [[1 3 2 3]] True Y:  [[1, 2, 2, 3]]
7 loss: 0.8428371 prediction:  [[1 3 2 3]] True Y:  [[1, 2, 2, 3]]
8 loss: 0.7969122 prediction:  [[1 3 2 3]] True Y:  [[1, 2, 2, 3]]
9 loss: 0.7607961 prediction:  [[1 3 2 3]] True Y:  [[1, 2, 2, 3]]
10 loss: 0.7152057 prediction:  [[1 3 2 3]] True Y:  [[1, 2, 2, 3]]
11 loss: 0.66892797 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
12 loss: 0.63111866 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
13 loss: 0.60339355 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
14 loss: 0.583441 prediction:  [[1 2 2 3]] True Y:  [[1,

121 loss: 0.3886088 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
122 loss: 0.38841996 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
123 loss: 0.38823342 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
124 loss: 0.3880489 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
125 loss: 0.3878662 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
126 loss: 0.38768572 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
127 loss: 0.38750684 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
128 loss: 0.3873303 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
129 loss: 0.38715523 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
130 loss: 0.3869822 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
131 loss: 0.38681087 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
132 loss: 0.38664138 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
133 loss: 0.38647372 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
134 loss: 0.38630757 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
135 loss: 0.38614333 pred

350 loss: 0.3689536 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
351 loss: 0.36891282 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
352 loss: 0.36887226 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
353 loss: 0.3688319 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
354 loss: 0.36879164 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
355 loss: 0.36875153 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
356 loss: 0.36871165 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
357 loss: 0.3686719 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
358 loss: 0.36863232 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
359 loss: 0.36859292 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
360 loss: 0.36855364 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
361 loss: 0.3685146 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
362 loss: 0.36847565 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
363 loss: 0.3684369 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
364 loss: 0.36839828 pred

572 loss: 0.36270192 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
573 loss: 0.36268222 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
574 loss: 0.36266255 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
575 loss: 0.36264294 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
576 loss: 0.3626234 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
577 loss: 0.36260384 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
578 loss: 0.36258438 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
579 loss: 0.36256498 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
580 loss: 0.3625456 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
581 loss: 0.3625263 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
582 loss: 0.36250705 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
583 loss: 0.3624878 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
584 loss: 0.3624686 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
585 loss: 0.36244953 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
586 loss: 0.36243045 pred

793 loss: 0.3592877 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
794 loss: 0.35927463 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
795 loss: 0.35926205 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
796 loss: 0.35924977 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
797 loss: 0.35923773 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
798 loss: 0.35922587 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
799 loss: 0.35921413 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
800 loss: 0.3592025 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
801 loss: 0.35919112 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
802 loss: 0.3591802 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
803 loss: 0.35916954 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
804 loss: 0.35916018 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
805 loss: 0.35915107 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
806 loss: 0.35914612 prediction:  [[1 2 2 3]] True Y:  [[1, 2, 2, 3]]
807 loss: 0.35913983 pr

참조링크 : <a href='https://gomguard.tistory.com/145'>argmax의 axis=0,1,2에 따른 이해</a>